# EDA

In [1]:
import numpy as np
import pandas as pd

In [3]:
feat = pd.read_csv('features.csv', header=None)
feat.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,195,72,133,246,117,72,72,199,68,36,...,21,0,0,0,0,128,250,226,116,221
1,239,159,239,144,147,0,0,128,147,0,...,0,144,147,0,0,128,147,0,0,64
2,229,32,32,132,226,8,0,81,227,8,...,32,131,5,22,0,0,10,80,0,147
3,3,0,0,0,196,32,96,0,3,0,...,160,4,198,32,96,0,196,0,160,8
4,20,64,0,60,0,0,0,0,131,194,...,24,16,0,0,24,18,0,67,16,42


In [9]:
feat.shape

(238982, 64)

In [8]:
lbl = pd.read_csv('labels.csv', header=None)
lbl.head()

,0
0,x86_64
1,avr
2,arm
3,sparc
4,mips


In [10]:
lbl.shape

(238982, 1)

In [28]:
X = feat.values
y = lbl.values.reshape(-1)

---

# Test models

## Split data

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

## Pre-processing

In [37]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler

### Scale X

In [43]:
scaler = StandardScaler()

In [75]:
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Encode y

In [45]:
le = LabelEncoder()

In [76]:
y_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

## Models

In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [60]:
clf1 = LogisticRegression(random_state=42).fit(X_scaled, y_enc)

In [77]:
clf1.score(X_test_scaled, y_test_enc)

0.22967095669815507

In [69]:
clf2 = DecisionTreeClassifier(random_state=42).fit(X_scaled, y_enc)

In [78]:
clf2.score(X_test_scaled, y_test_enc)

0.7475305902491599

In [73]:
clf3 = RandomForestClassifier(random_state=42).fit(X_scaled, y_enc)

In [79]:
clf3.score(X_test_scaled, y_test_enc)

0.9610473594116529